# Our project

In [3]:
import nltk

In [5]:
partial_grammar = """
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
"""
title = "British scientist says memories of spending the night with Marilyn Monroe could be implanted into the brain"
title_words = nltk.word_tokenize(title)
title_pos = nltk.pos_tag(title_words)
print title_pos

[('British', 'JJ'), ('scientist', 'NN'), ('says', 'VBZ'), ('memories', 'NNS'), ('of', 'IN'), ('spending', 'VBG'), ('the', 'DT'), ('night', 'NN'), ('with', 'IN'), ('Marilyn', 'NNP'), ('Monroe', 'NNP'), ('could', 'MD'), ('be', 'VB'), ('implanted', 'VBN'), ('into', 'IN'), ('the', 'DT'), ('brain', 'NN')]


In [18]:
def lex_grammar(words_pos):
    lexicon = {}
    for word, pos in sorted(words_pos, key=lambda (a, b): b):
        if pos not in lexicon:
            lexicon[pos] = []
        lexicon[pos].append(word)
    return "\n".join("{pos} -> {words}".format(pos=pos, words="|".join(ws)) for pos, ws in lexicon.iteritems())

In [19]:
second_part_of_grammar = lex_grammar(title_pos)
grammar = partial_grammar + second_part_of_grammar

In [20]:
print grammar


S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
MD -> could
VB -> be
VBG -> spending
NN -> scientist|night|brain
VBN -> implanted
JJ -> British
IN -> of|with|into
VBZ -> says
DT -> the|the
NNS -> memories
NNP -> Marilyn|Monroe
